In [1]:
import pandas as pd
import numpy as np
import openpyxl # Para abrir excel
import os
from datetime import datetime, timedelta

In [2]:
# Fijamos el directorio de trabajo
DIRECTORY =  "/home/spperez/masiv_model/"

In [3]:
# Se fija el formato de InvoiceDate para no tener problemas
df = pd.read_excel(io = DIRECTORY + "data/raw/online_retail.xlsx", 
        sheet_name="Online Retail", converters={'InvoiceDate':str})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Preparación de los Datos

Se averegua la dimension de los datos, su esquema y la existencia de datos faltantes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# La variable InvoiceDate se pasa de string a fecha
df["InvoiceDate"] = df["InvoiceDate"].apply(lambda x: x.split(' ')[0])
df["InvoiceDate"] = df["InvoiceDate"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df["InvoiceDate"].head()

0   2010-12-01
1   2010-12-01
2   2010-12-01
3   2010-12-01
4   2010-12-01
Name: InvoiceDate, dtype: datetime64[ns]

La base de datos consta de 551909 registros y 8 varables.
El esquema está compuesto por varaibles numericas de tipo entero y tipo flotante. También hay variables con tipo objeto y fecha. 
 
Se observa que hay dos variables con datos faltantes, CustomerID y Description del producto. 

In [6]:
# NaN en porcentajes 
rows, columns = df.shape
cell_count = rows * columns
number_of_nulls = df.isnull().sum().sum()
percentage_of_missing = (number_of_nulls / cell_count) * 100
print(f"Se tiene el {percentage_of_missing.round(2) }% de valores perdidos.")

Se tiene el 3.15% de valores perdidos.


In [7]:
df.isnull().sum()/rows*100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

Del total de los registros exitentes en la base de datos, el 3.5% están vacios. En la variable CustomerId el 25% de los regitros están imcompletos y en la variable Description el 0.2% , una sifra bastante baja. 
Los análisis que se van a relalizar más adelante seran a nivel cliente y por tanto requiere que cada transacción este asociada a uno. Antes de descartar los registros cuyo valor en CustomerID está vacio, veamos si le podemos asociar un cliente.

In [8]:
# Registos sin cliente
df_nul = df.iloc[df[df['CustomerID'].isnull()].index.tolist()]
# Retiramos esos registros del conjunto original
df = df[~df.index.isin(df_nul.index)]
df.reset_index(inplace=True, drop=True)
# Le podemos asociar un cliente exitente?
df_nul["InvoiceNo"].isin(df["InvoiceNo"]).sum()

0

No hay ninguna coincidencia entre los registros sin cliente y los completos, por tanto no se pueden llenar. Esos registros se retiran del conjunto de datos y se pasa a tener 406829 registros.

In [9]:
df.describe(include="O")

,InvoiceNo,StockCode,Description,Country
count,406829,406829,406829,406829
unique,22190,3684,3896,37
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,542,2077,2070,361878


In [10]:
len(df["CustomerID"].unique())

4372

In [11]:
df.describe(datetime_is_numeric=True)

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,406829.000000,406829,406829.000000,406829.000000
mean,12.061303,2011-07-10 03:16:19.124890624,3.460471,15287.690570
min,-80995.000000,2010-12-01 00:00:00,0.000000,12346.000000
25%,2.000000,2011-04-06 00:00:00,1.250000,13953.000000
50%,5.000000,2011-07-31 00:00:00,1.950000,15152.000000
75%,12.000000,2011-10-20 00:00:00,3.750000,16791.000000
max,80995.000000,2011-12-09 00:00:00,38970.000000,18287.000000
std,248.693370,NaN,69.315162,1713.600303


La base de datos despues de retirar los registros con valores faltantes en la variable CustomerID, queda con 406829 registros con información de 22190 transacciones, hechas por 4372 clientes de 37 paises distintos entre 2010-12-01 y 2011-12-09. Se vendieron 3684 productos diferentes.

En el resumen anterior muetra que existen valores negativos en la variable Quantity. Estos valores se pueden dar por dos razones: 

- Los clientes hacen algun tipo de devolución
- Errores en el registro de la información
- Ajustes en la transacción

Para poder retirar esto valores negativos es necesario agregar la base de datos a nivel de cliente, porque en caso de devoluciones, estos valores se contrarrestan dejando así al cliente con cantidad comprada igual a cero y por tanto ese cliente se debe retirar del analisis.

In [12]:
df[(df["Quantity"] < 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
406377,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09,0.83,14397.0,United Kingdom
406461,C581499,M,Manual,-1,2011-12-09,224.69,15498.0,United Kingdom
406635,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09,10.95,15311.0,United Kingdom
406636,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09,1.25,17315.0,United Kingdom


No hay forma de identificar cuál es la razón de los valores negativos en las cantidades, por tanto retiramos estos registros de la base de datos y asumimos que el cliente sí hizo la compra.

In [13]:
df = df[df["Quantity"] > 0]
df = df[df["UnitPrice"] > 0]
df.describe(datetime_is_numeric=True)

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,397884.000000,397884,397884.000000,397884.000000
mean,12.988238,2011-07-10 10:27:17.518472704,3.116488,15294.423453
min,1.000000,2010-12-01 00:00:00,0.001000,12346.000000
25%,2.000000,2011-04-07 00:00:00,1.250000,13969.000000
50%,6.000000,2011-07-31 00:00:00,1.950000,15159.000000
75%,12.000000,2011-10-20 00:00:00,3.750000,16795.000000
max,80995.000000,2011-12-09 00:00:00,8142.750000,18287.000000
std,179.331775,NaN,22.097877,1713.141560


In [14]:
df.describe(include="O")

,InvoiceNo,StockCode,Description,Country
count,397884,397884,397884,397884
unique,18532,3665,3877,37
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,542,2035,2028,354321


In [15]:
len(df["CustomerID"].unique())

4338

In [16]:
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


Despues de limpiar completamenta la base de datos queda con 397884 registros con información de 18532 transacciones, hechas por 4338 clientes de 37 paises distintos entre 2010-12-01 y 2011-12-09. Se vendieron 3665 productos diferentes.

In [33]:
df.to_csv(DIRECTORY + 'data/interim/clean_data.csv', index=False)

### Transacciones y Ventas Semanales de la Tienda

Se sacan estos datos para hacer el modelo de predicción de la venta y las transacciones del próximo periodo.

In [43]:
# Se cargan datos externos de las semanas y los meses
df_dates = pd.read_csv(DIRECTORY + 'data/external/dates.csv', sep=";")
df_dates["DAYDT"] = df_dates["DAYDT"].apply(lambda x: x.split(' ')[0])
df_dates["DAYDT"] = df_dates["DAYDT"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df_dates.head()

,DAYDT,WEEKOFYEARID,YEARID,MONTH
0,2010-05-29,21,2010,5
1,2010-12-19,50,2010,12
2,2010-11-22,47,2010,11
3,2012-06-12,24,2012,6
4,2011-09-25,38,2011,9


In [67]:
# Datos monetarios semanales
df_week = df.merge(df_dates, how="left", left_on="InvoiceDate", right_on="DAYDT")
df_week.sort_values(by=["InvoiceDate"], ascending=True, inplace=True)
df_week = df_week.groupby(["YEARID", "WEEKOFYEARID"])["OrderValue"].sum().reset_index()
df_week["YearWeek"] = df_week["YEARID"].astype(str) + "-" + df_week["WEEKOFYEARID"].astype(str)
df_week.rename(columns={"OrderValue": "Monetary"}, inplace=True)
df_week

,YEARID,WEEKOFYEARID,Monetary,YearWeek
0,2010,48,149386.330,2010-48
1,2010,49,213447.720,2010-49
2,2010,50,163770.730,2010-50
3,2010,51,46109.110,2010-51
4,2011,1,114865.270,2011-1
5,2011,2,154714.940,2011-2
6,2011,3,175757.980,2011-3
7,2011,4,105288.770,2011-4
8,2011,5,106095.230,2011-5
9,2011,6,88015.420,2011-6


In [68]:
# Datos transacciones semanales
df_week_transa = df.merge(df_dates, how="left", left_on="InvoiceDate", right_on="DAYDT")
df_week_transa.sort_values(by=["InvoiceDate"], ascending=True, inplace=True)
df_week_transa = df_week_transa.groupby(["YEARID", "WEEKOFYEARID"])[["InvoiceNo"]].nunique().reset_index()
df_week_transa["YearWeek"] = df_week_transa["YEARID"].astype(str) + "-" + df_week_transa["WEEKOFYEARID"].astype(str)
df_week_transa.rename(columns={"InvoiceNo": "Transactions"}, inplace=True)
df_week_transa.head(10)

,YEARID,WEEKOFYEARID,Transactions,YearWeek
0,2010,48,402,2010-48
1,2010,49,489,2010-49
2,2010,50,401,2010-50
3,2010,51,108,2010-51
4,2011,1,224,2011-1
5,2011,2,233,2011-2
6,2011,3,206,2011-3
7,2011,4,267,2011-4
8,2011,5,272,2011-5
9,2011,6,197,2011-6


In [69]:
# Se guardan los datos en la carperta interim
df_week.to_csv(DIRECTORY + 'data/interim/monetary_value_week.csv', index=False)
df_week_transa.to_csv(DIRECTORY + 'data/interim/transactions_week.csv', index=False)

### Transacciones y Ventas Mensuales de la Tienda

In [53]:
# Datos monetarios semanales
df_month = df.groupby("Month")["OrderValue"].sum().reset_index()
df_month.rename(columns={"OrderValue": "Monetary"}, inplace=True)
df_month.head()

,Month,Monetary
0,2010-12,572713.890
1,2011-01,569445.040
2,2011-02,447137.350
3,2011-03,595500.760
4,2011-04,469200.361


In [54]:
# Datos transacciones Mensuales
df_month_transa = df.groupby("Month")[["InvoiceNo"]].nunique().reset_index()
df_month_transa.rename(columns={"InvoiceNo": "Transactions"}, inplace=True)
df_month_transa.head()

,Month,Transactions
0,2010-12,1400
1,2011-01,987
2,2011-02,997
3,2011-03,1321
4,2011-04,1149


In [55]:
# Se guardan los datos en la carperta interim
df_month.to_csv(DIRECTORY + 'data/interim/monetary_value_month.csv', index=False)
df_month_transa.to_csv(DIRECTORY + 'data/interim/transactions_month.csv', index=False)